In [2]:

# PRAW, a Python library, to interact with Reddit API.
import praw
#install TextBlob library for processing textual data andsentiment analysis
from textblob import TextBlob
import nltk
# Download VADER, VADER (Valence Aware Dictionary and sEntiment Reasoner) a rule-based sentiment analysis tool
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# create object for VADER sentiment function interaction
sia = SentimentIntensityAnalyzer()

reddit = praw.Reddit(client_id='ThmLaDEf6e3BZA',
                     client_secret='n1LJbrjudo8m0G5UMiSosguAyOg',
                     user_agent='SentScrape')
#create an app and get the required authentication and details.These are mine
# get 10 hot posts from the showerthoughts subreddit
top_posts = reddit.subreddit('creativecloud').top('all', limit=30)
#Replace creativecloud with any subreddit name and replace year with either week,month,day or year
# Sentiment analysis function for TextBlob tools
def text_blob_sentiment(review, sub_entries_textblob):
    analysis = TextBlob(review)
    if analysis.sentiment.polarity >= 0.0001:
        if analysis.sentiment.polarity > 0:
            sub_entries_textblob['positive'] = sub_entries_textblob['positive'] + 1
            return 'Positive'

    elif analysis.sentiment.polarity <= -0.0001:
        if analysis.sentiment.polarity <= 0:
            sub_entries_textblob['negative'] = sub_entries_textblob['negative'] + 1
            return 'Negative'
    else:
        sub_entries_textblob['neutral'] = sub_entries_textblob['neutral'] + 1
        return 'Neutral'
    

# sentiment analysis function for VADER tool
def nltk_sentiment(review, sub_entries_nltk):
    vs = sia.polarity_scores(review)
    if not vs['neg'] > 0.05:
        if vs['pos'] - vs['neg'] > 0:
            sub_entries_nltk['positive'] = sub_entries_nltk['positive'] + 1
            return 'Positive'
        else:
            sub_entries_nltk['neutral'] = sub_entries_nltk['neutral'] + 1
            return 'Neutral'

    elif not vs['pos'] > 0.05:
        if vs['pos'] - vs['neg'] <= 0:
            sub_entries_nltk['negative'] = sub_entries_nltk['negative'] + 1
            return 'Negative'
        else:
            sub_entries_nltk['neutral'] = sub_entries_nltk['neutral'] + 1
            return 'Neutral'
    else:
        sub_entries_nltk['neutral'] = sub_entries_nltk['neutral'] + 1
        return 'Neutral'


# replication of comment section of reddit post
def replies_of(top_level_comment, count_comment, sub_entries_textblob, sub_entries_nltk):
    if len(top_level_comment.replies) == 0:
        count_comment = 0
        return
    else:
        for num, comment in enumerate(top_level_comment.replies):
            try:
                count_comment += 1
                print('-' * count_comment, comment.body)
                text_blob_sentiment(comment.body, sub_entries_textblob)
                nltk_sentiment(comment.body, sub_entries_nltk)
            except:
                continue
            replies_of(comment, count_comment, sub_entries_textblob,sub_entries_nltk)


def main():
    
    for submission in top_posts:
        sub_entries_textblob = {'negative': 0, 'positive' : 0, 'neutral' : 0}
        sub_entries_nltk = {'negative': 0, 'positive' : 0, 'neutral' : 0}
        print('Title of the post :', submission.title)
        text_blob_sentiment(submission.title, sub_entries_textblob)
        nltk_sentiment(submission.title, sub_entries_nltk)
        print("\n")
        submission_comm = reddit.submission(id=submission.id)

        for count, top_level_comment in enumerate(submission_comm.comments):
            print(f"-------------{count} top level comment start--------------")
            count_comm = 0
            try :
                print(top_level_comment.body)
                text_blob_sentiment(top_level_comment.body, sub_entries_textblob)
                nltk_sentiment(top_level_comment.body, sub_entries_nltk)
                replies_of(top_level_comment,
                           count_comm,
                           sub_entries_textblob,
                           sub_entries_nltk)
            except:
                continue
        print('Over all Sentiment of Topic by TextBlob :', sub_entries_textblob)
        print('Over all Sentiment of Topic by VADER :', sub_entries_nltk)
        print("\n\n\n")

if __name__ == '__main__' :
    main()

Title of the post : CC Rainbow (i installed bridge just for this)


-------------0 top level comment start--------------
Man that Photoshop logo really bothers me
- Same bro, don’t know why I didn’t replace it with something else
Over all Sentiment of Topic by TextBlob : {'negative': 0, 'positive': 1, 'neutral': 2}
Over all Sentiment of Topic by VADER : {'negative': 1, 'positive': 0, 'neutral': 2}




Title of the post : Creative Cloud Waving Monthly Fees for 2 Months


-------------0 top level comment start--------------
You’re the real MVP.
-------------1 top level comment start--------------
What if I have pre paid for 12 months?
- Doesn't apply I believe.
-- Yeah, sadly. I tried but no luck
-------------2 top level comment start--------------
Watch out though, Adobe's pretty greedy and the deals you find available in two months might not be the deal you have today. They've been surveying users to find out what pairing of price and features they'll respond positively to, and all of 

-------------0 top level comment start--------------
i was questioning how in the hell you have an account named just mike but i saw it was 12 years old now.. 
-------------1 top level comment start--------------
Isn't 29.99 the student promotional rate?   Maybe when you signed up , you entered an expected graduation date and when that date came around it auto-upped to the regular membership along with regular membership price? 
- That’s what happened to me. Over time I lost the benefits and they just switched me up to the higher price without letting me know
-- That could be it. 
-------------2 top level comment start--------------
I want to know how you've been getting the $29.99/mo 1-year promo deal for four years.
- ????
-- The deal adobe offers is $30 for an introductory year. Then it goes up to $50. You got that deal from 2014-2018. How?!
--- No clue
-------------3 top level comment start--------------
I received emails about 2-3 months ahead of the renewal so you may want to tri

Over all Sentiment of Topic by TextBlob : {'negative': 0, 'positive': 1, 'neutral': 0}
Over all Sentiment of Topic by VADER : {'negative': 0, 'positive': 1, 'neutral': 0}




Title of the post : Adobe Creative Cloud for retired designers and seniors


-------------0 top level comment start--------------
I think that’s a great idea
-------------1 top level comment start--------------
Yes! Do it, so it’s in place when I retire.
-------------2 top level comment start--------------
Retired designer here too! I still freelance part-time so subscribe to CC, but it would help a lot on a more modest income to have a student-type discount. So yes, I'm on board with you!
Over all Sentiment of Topic by TextBlob : {'negative': 0, 'positive': 3, 'neutral': 1}
Over all Sentiment of Topic by VADER : {'negative': 0, 'positive': 4, 'neutral': 0}




Title of the post : Adobe Max Sneak Peek - Editing voice over by typing new words!


-------------0 top level comment start--------------
Clearly needs a l

-------------0 top level comment start--------------
Pro tip: just commit fraud if you don't feel like fulfilling a legal obligation
- Pro tip:  Just don't use Adobe's scamming subscription service.
-- But it is not fraud when a rep for them lies, offers free months and says “there is no cancellation fee”? I see a double standard here.
--- What made you think to use a disposable credit card number for Creative Cloud? How do you subscribe via conversation with a sales rep? Curious to hear your story.
-------------1 top level comment start--------------
Wow! This looks incredibly promising. Can’t believe traditional CC companies don’t have these features built in to begin with.
- Um, because banks have an agreement with these large companies... 💁‍♀️
-------------2 top level comment start--------------
[deleted]
- While I agree, to play devils advocate these contracts aren't exactly in plan language, easy to easy or understand. 💁‍♀️
-- [deleted]
--- Actually, that's not been as tested as 

-------------0 top level comment start--------------
I don't know if this is true for all apps, but it's a significant change from last time. I assume this is because everyone was complaining they had multiple versions last year and 'why doesn't it update in place' etc?

Personally I'd suggest if you have any ongoing work in 2014 you keep it around. There are always bugs/kinks/changes that can cause problems, not to mention plugins that no longer work etc.

Still guess it does streamline the experience...
-------------1 top level comment start--------------
It seems the after effects team are recommending keeping the old AE around. Their impressive re-architecture of the rendering engine has changed things in ways affecting most plugins. They have a list of a few 'big names' still not working [here]( plug-ins and After Effects CC 2015 (13.5)
http://blogs.adobe.com/aftereffects/2015/06/plug-ins-and-after-effects-cc-2015-13-5.html) although they warn other plugins may also need to be upd

Over all Sentiment of Topic by TextBlob : {'negative': 0, 'positive': 0, 'neutral': 1}
Over all Sentiment of Topic by VADER : {'negative': 0, 'positive': 0, 'neutral': 1}




